In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
# from torchsummary import summary

import os
import time
import datetime
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.legend_handler import HandlerPathCollection


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1_1 = nn.Conv2d(1, 32, 5, 1, 2) # in, out, kernel, stride, padding 
        self.prelu1_1 = nn.PReLU()
        self.conv1_2 = nn.Conv2d(32, 32, 5, 1, 2)
        self.prelu1_2 = nn.PReLU()

        self.conv2_1 = nn.Conv2d(32, 64, 5, 1, 2)
        self.prelu2_1 = nn.PReLU()
        self.conv2_2 = nn.Conv2d(64, 64, 5, 1, 2)
        self.prelu2_2 = nn.PReLU()

        self.conv3_1 = nn.Conv2d(64, 128, 5, 1, 2)
        self.prelu3_1 = nn.PReLU()
        self.conv3_2 = nn.Conv2d(128, 128, 5, 1, 2)
        self.prelu3_2 = nn.PReLU()

        self.fc1 = nn.Linear(128 * 3 * 3, 2)
        self.prelu_fc1 = nn.PReLU()
        self.fc2 = nn.Linear(2, 10)

    def forward(self, x):
        # Stage 1
        x = self.prelu1_1(self.conv1_1(x)) # 28
        x = self.prelu1_2(self.conv1_2(x)) # 28
        x = F.max_pool2d(x, 2, 2, 0) # 14

        # Stage 2
        x = self.prelu2_1(self.conv2_1(x)) # 14
        x = self.prelu2_2(self.conv2_2(x)) # 14
        x = F.max_pool2d(x, 2, 2, 0) # 7

        # Stage 3
        x = self.prelu3_1(self.conv3_1(x)) # 7
        x = self.prelu3_2(self.conv3_2(x)) # 7
        x = F.max_pool2d(x, 2, 2, 0) # 3

        # Flatten
        # x = torch.flatten(x, 1)
        # x = x.view(x.size(0), -1)
        x = x.view(-1, 128 * 3 * 3)
        x = self.prelu_fc1(self.fc1(x))
        y = self.fc2(x)
        
        return x, y

In [2]:
use_cuda = 'cuda'
# device = torch.device("cuda" if use_cuda else "cpu")
device = 'cuda:3'

lenet = Net().to(device=device)
# summary(lenet, input_size=(1, 28, 28))

In [3]:
batch_size = 128
test_batch_size = 1000
epochs = 50
lr = 1e-2
step_size = 20
gamma = 0.7
log_interval = 10
save_model = True
num_classes = 10
center_loss_weight = 0.001

loss_opt = 'softmax_center' # 'softmax' or 'softmax_center'

class CenterLoss(nn.Module):
    def __init__(self, num_classes=10, feat_dim=2):
        super(CenterLoss, self).__init__()
        self.num_classes = num_classes
        self.feat_dim = feat_dim
        self.centers = nn.Parameter(torch.randn(self.num_classes, self.feat_dim).cuda())

    def forward(self, x, labels):
        """
        Args:
            x: feature matrix with shape (batch_size, feat_dim).
            labels: ground truth labels with shape (batch_size).
        """
        center = self.centers[labels]
        dist = (x - center).pow(2).sum(dim=-1) / 2.0
        loss = torch.clamp(dist, min=1e-12, max=1e+12).mean(dim=-1)

        return loss

def plot_features(features, labels, num_classes, epoch, prefix):
    """Plot features on 2D plane.
    Args:
        features: (num_instances, num_features).
        labels: (num_instances). 
    """
    colors = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
    for label_idx in range(num_classes):
        sc = plt.scatter(
            features[labels==label_idx, 0],
            features[labels==label_idx, 1],
            c=colors[label_idx],
            s=1,
        )
    plt.legend(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], bbox_to_anchor=(1.04, 1), loc='upper left', 
               handler_map={type(sc): HandlerPathCollection(update_func=update_prop)}) # prop={'size': 6}
    # dirname = osp.join(args.save_dir, prefix)
    # if not osp.exists(dirname):
    #     os.mkdir(dirname)
    save_name = prefix + '_epoch_' + str(epoch) + '.png'
    plt.savefig(save_name, bbox_inches='tight')
    plt.close()
    

def update_prop(handle, orig):
    marker_size = 12
    handle.update_from(orig)
    handle.set_sizes([marker_size])
    
    
def crossentropyloss_train(model, criterion_cross, device, train_loader, optimizer_cross, epoch, num_classes, log_interval):
    model.train()
    all_features, all_labels = [], []

    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        feature, output = model(data)
        loss = criterion_cross(output, label)

        optimizer_cross.zero_grad()
        loss.backward()

        optimizer_cross.step()

        all_features.append(feature.data.cpu().numpy())
        all_labels.append(label.data.cpu().numpy())

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            # if args.dry_run:
            #     break

    all_features = np.concatenate(all_features, 0)
    all_labels = np.concatenate(all_labels, 0)
    plot_features(all_features, all_labels, num_classes, epoch, prefix='softmax')


def crosscenterloss_train(model, criterion_cross, criterion_center, device, train_loader, optimizer_cross, optimizer_center, epoch, num_classes, log_interval, weight=1):
    model.train()
    all_features, all_labels = [], []

    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        feature, output = model(data)
        loss_cross = criterion_cross(output, label)
        loss_center = criterion_center(feature, label)
        loss_center *= weight
        loss = loss_center + loss_cross

        optimizer_cross.zero_grad()
        optimizer_center.zero_grad()
        loss.backward()

        optimizer_cross.step()
        # by doing so, weight_cent would not impact on the learning of centers
        for param in criterion_center.parameters():
            param.grad.data *= (1. / weight)
        optimizer_center.step()

        all_features.append(feature.data.cpu().numpy())
        all_labels.append(label.data.cpu().numpy())

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tCenter Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item(), loss_center.item()))
            # if args.dry_run:
            #     break

    all_features = np.concatenate(all_features, 0)
    all_labels = np.concatenate(all_labels, 0)
    plot_features(all_features, all_labels, num_classes, epoch, prefix=f'softmax_center_lambda_{weight}') # soft_center

    
def test(model, criterion, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            feature, output = model(data)
            # test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


# Training settings
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.Resize((28,28)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

# transform = transforms.Compose([
#     transforms.Resize((28,28)),
#     transforms.ToTensor()
# ])

dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)

with torch.cuda.device(3):
    criterion_cross = nn.CrossEntropyLoss()
    optimizer_cross = optim.SGD(model.parameters(), lr=lr, weight_decay=5e-4, momentum=0.9) # weight_decay=5e-4, momentum=0.9
    
    if loss_opt == 'softmax_center':
        criterion_center = CenterLoss(num_classes=num_classes, feat_dim=2)
        optimizer_center = optim.SGD(criterion_center.parameters(), lr=lr)
        
scheduler = StepLR(optimizer_cross, step_size=step_size, gamma=gamma)

In [4]:
start_time = time.time()
with torch.cuda.device(3):
    if loss_opt == 'softmax':
        for epoch in range(1, epochs + 1):
            crossentropyloss_train(model, criterion_cross, device, train_loader, 
                optimizer_cross, epoch, num_classes, log_interval)
            test(model, criterion_cross, device, test_loader)
            scheduler.step()
    elif loss_opt == 'softmax_center':
        for epoch in range(1, epochs + 1):
            crosscenterloss_train(model, criterion_cross, criterion_center, device, train_loader, 
                optimizer_cross, optimizer_center, epoch, num_classes, log_interval, center_loss_weight)
            test(model, criterion_cross, device, test_loader)
            scheduler.step()

elapsed = round(time.time() - start_time)
elapsed = str(datetime.timedelta(seconds=elapsed))
print("Finished. Total elapsed time (h:m:s): {}".format(elapsed))

if save_model:
    torch.save(model.state_dict(), "mnist_cnn.pt")


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.369788	Center Loss: 0.001251
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.327131	Center Loss: 0.001038
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.383770	Center Loss: 0.001148
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.281431	Center Loss: 0.001165
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.301089	Center Loss: 0.001482
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.246783	Center Loss: 0.001415
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.203568	Center Loss: 0.001288
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.977149	Center Loss: 0.002882
Train Epoch: 1 [10240/60000 (17%)]	Loss: 1.675945	Center Loss: 0.007647
Train Epoch: 1 [11520/60000 (19%)]	Loss: 1.487436	Center Loss: 0.009293
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.363423	Center Loss: 0.013846
Train Epoch: 1 [14080/60000 (23%)]	Loss: 1.206473	Center Loss: 0.023728
Train Epoch: 1 [15360/60000 (26%)]	Loss: 1.100639	Center Loss: 0.036695
Train Epoch: 1 [16640/60000 (28%)]	Loss: 1.130085	Center Loss: 0.025970
Train Ep

Train Epoch: 3 [24320/60000 (41%)]	Loss: 0.137410	Center Loss: 0.032943
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.155696	Center Loss: 0.015110
Train Epoch: 3 [26880/60000 (45%)]	Loss: 0.112868	Center Loss: 0.016779
Train Epoch: 3 [28160/60000 (47%)]	Loss: 0.128147	Center Loss: 0.012652
Train Epoch: 3 [29440/60000 (49%)]	Loss: 0.142776	Center Loss: 0.019333
Train Epoch: 3 [30720/60000 (51%)]	Loss: 0.177220	Center Loss: 0.016477
Train Epoch: 3 [32000/60000 (53%)]	Loss: 0.156995	Center Loss: 0.006273
Train Epoch: 3 [33280/60000 (55%)]	Loss: 0.151276	Center Loss: 0.020093
Train Epoch: 3 [34560/60000 (58%)]	Loss: 0.116541	Center Loss: 0.012526
Train Epoch: 3 [35840/60000 (60%)]	Loss: 0.109002	Center Loss: 0.010216
Train Epoch: 3 [37120/60000 (62%)]	Loss: 0.125657	Center Loss: 0.017211
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.168154	Center Loss: 0.011044
Train Epoch: 3 [39680/60000 (66%)]	Loss: 0.418111	Center Loss: 0.020945
Train Epoch: 3 [40960/60000 (68%)]	Loss: 0.096622	Center Loss: 0

Train Epoch: 5 [48640/60000 (81%)]	Loss: 0.046934	Center Loss: 0.005398
Train Epoch: 5 [49920/60000 (83%)]	Loss: 0.038428	Center Loss: 0.006921
Train Epoch: 5 [51200/60000 (85%)]	Loss: 0.127136	Center Loss: 0.009338
Train Epoch: 5 [52480/60000 (87%)]	Loss: 0.048747	Center Loss: 0.011049
Train Epoch: 5 [53760/60000 (90%)]	Loss: 0.099815	Center Loss: 0.005716
Train Epoch: 5 [55040/60000 (92%)]	Loss: 0.036968	Center Loss: 0.010706
Train Epoch: 5 [56320/60000 (94%)]	Loss: 0.093020	Center Loss: 0.006915
Train Epoch: 5 [57600/60000 (96%)]	Loss: 0.096466	Center Loss: 0.005831
Train Epoch: 5 [58880/60000 (98%)]	Loss: 0.089536	Center Loss: 0.013083

Test set: Average loss: 0.0001, Accuracy: 9846/10000 (98%)

Train Epoch: 6 [0/60000 (0%)]	Loss: 0.047063	Center Loss: 0.007886
Train Epoch: 6 [1280/60000 (2%)]	Loss: 0.061258	Center Loss: 0.007046
Train Epoch: 6 [2560/60000 (4%)]	Loss: 0.046273	Center Loss: 0.009554
Train Epoch: 6 [3840/60000 (6%)]	Loss: 0.047319	Center Loss: 0.006934
Train Epoch: 6

Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.058634	Center Loss: 0.004544
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.034603	Center Loss: 0.003908
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.023231	Center Loss: 0.005719
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.018389	Center Loss: 0.006615
Train Epoch: 8 [16640/60000 (28%)]	Loss: 0.030822	Center Loss: 0.004211
Train Epoch: 8 [17920/60000 (30%)]	Loss: 0.018012	Center Loss: 0.006331
Train Epoch: 8 [19200/60000 (32%)]	Loss: 0.021816	Center Loss: 0.004741
Train Epoch: 8 [20480/60000 (34%)]	Loss: 0.024367	Center Loss: 0.006115
Train Epoch: 8 [21760/60000 (36%)]	Loss: 0.020940	Center Loss: 0.003097
Train Epoch: 8 [23040/60000 (38%)]	Loss: 0.021068	Center Loss: 0.005492
Train Epoch: 8 [24320/60000 (41%)]	Loss: 0.035390	Center Loss: 0.005846
Train Epoch: 8 [25600/60000 (43%)]	Loss: 0.022009	Center Loss: 0.004660
Train Epoch: 8 [26880/60000 (45%)]	Loss: 0.038326	Center Loss: 0.003929
Train Epoch: 8 [28160/60000 (47%)]	Loss: 0.036117	Center Loss: 0

Train Epoch: 10 [37120/60000 (62%)]	Loss: 0.013426	Center Loss: 0.005413
Train Epoch: 10 [38400/60000 (64%)]	Loss: 0.009449	Center Loss: 0.003422
Train Epoch: 10 [39680/60000 (66%)]	Loss: 0.025442	Center Loss: 0.004239
Train Epoch: 10 [40960/60000 (68%)]	Loss: 0.015748	Center Loss: 0.004861
Train Epoch: 10 [42240/60000 (70%)]	Loss: 0.024254	Center Loss: 0.004183
Train Epoch: 10 [43520/60000 (72%)]	Loss: 0.011271	Center Loss: 0.003785
Train Epoch: 10 [44800/60000 (75%)]	Loss: 0.010041	Center Loss: 0.002725
Train Epoch: 10 [46080/60000 (77%)]	Loss: 0.014080	Center Loss: 0.003639
Train Epoch: 10 [47360/60000 (79%)]	Loss: 0.010426	Center Loss: 0.002695
Train Epoch: 10 [48640/60000 (81%)]	Loss: 0.012474	Center Loss: 0.004254
Train Epoch: 10 [49920/60000 (83%)]	Loss: 0.011508	Center Loss: 0.004491
Train Epoch: 10 [51200/60000 (85%)]	Loss: 0.042007	Center Loss: 0.004698
Train Epoch: 10 [52480/60000 (87%)]	Loss: 0.013704	Center Loss: 0.005741
Train Epoch: 10 [53760/60000 (90%)]	Loss: 0.011738	

...

Train Epoch: 41 [25600/60000 (43%)]	Loss: 0.005577	Center Loss: 0.001480
Train Epoch: 41 [26880/60000 (45%)]	Loss: 0.006847	Center Loss: 0.002627
Train Epoch: 41 [28160/60000 (47%)]	Loss: 0.007054	Center Loss: 0.002416
Train Epoch: 41 [29440/60000 (49%)]	Loss: 0.005220	Center Loss: 0.001488
Train Epoch: 41 [30720/60000 (51%)]	Loss: 0.006176	Center Loss: 0.001974
Train Epoch: 41 [32000/60000 (53%)]	Loss: 0.005924	Center Loss: 0.001665
Train Epoch: 41 [33280/60000 (55%)]	Loss: 0.006950	Center Loss: 0.001877
Train Epoch: 41 [34560/60000 (58%)]	Loss: 0.006684	Center Loss: 0.002486
Train Epoch: 41 [35840/60000 (60%)]	Loss: 0.004855	Center Loss: 0.001667
Train Epoch: 41 [37120/60000 (62%)]	Loss: 0.005904	Center Loss: 0.001565
Train Epoch: 41 [38400/60000 (64%)]	Loss: 0.005656	Center Loss: 0.001513
Train Epoch: 41 [39680/60000 (66%)]	Loss: 0.005658	Center Loss: 0.001725
Train Epoch: 41 [40960/60000 (68%)]	Loss: 0.005933	Center Loss: 0.002017
Train Epoch: 41 [42240/60000 (70%)]	Loss: 0.004977	

Train Epoch: 43 [47360/60000 (79%)]	Loss: 0.005175	Center Loss: 0.001426
Train Epoch: 43 [48640/60000 (81%)]	Loss: 0.004488	Center Loss: 0.001564
Train Epoch: 43 [49920/60000 (83%)]	Loss: 0.005561	Center Loss: 0.002181
Train Epoch: 43 [51200/60000 (85%)]	Loss: 0.005672	Center Loss: 0.001448
Train Epoch: 43 [52480/60000 (87%)]	Loss: 0.004995	Center Loss: 0.002203
Train Epoch: 43 [53760/60000 (90%)]	Loss: 0.004772	Center Loss: 0.001423
Train Epoch: 43 [55040/60000 (92%)]	Loss: 0.004973	Center Loss: 0.001804
Train Epoch: 43 [56320/60000 (94%)]	Loss: 0.006295	Center Loss: 0.001756
Train Epoch: 43 [57600/60000 (96%)]	Loss: 0.006629	Center Loss: 0.002562
Train Epoch: 43 [58880/60000 (98%)]	Loss: 0.005837	Center Loss: 0.001907

Test set: Average loss: 0.0001, Accuracy: 9884/10000 (99%)

Train Epoch: 44 [0/60000 (0%)]	Loss: 0.005437	Center Loss: 0.001639
Train Epoch: 44 [1280/60000 (2%)]	Loss: 0.004825	Center Loss: 0.001528
Train Epoch: 44 [2560/60000 (4%)]	Loss: 0.004507	Center Loss: 0.001270

Train Epoch: 46 [8960/60000 (15%)]	Loss: 0.004949	Center Loss: 0.002158
Train Epoch: 46 [10240/60000 (17%)]	Loss: 0.004803	Center Loss: 0.001871
Train Epoch: 46 [11520/60000 (19%)]	Loss: 0.004844	Center Loss: 0.001346
Train Epoch: 46 [12800/60000 (21%)]	Loss: 0.004728	Center Loss: 0.001155
Train Epoch: 46 [14080/60000 (23%)]	Loss: 0.005020	Center Loss: 0.001340
Train Epoch: 46 [15360/60000 (26%)]	Loss: 0.004254	Center Loss: 0.001626
Train Epoch: 46 [16640/60000 (28%)]	Loss: 0.004209	Center Loss: 0.001425
Train Epoch: 46 [17920/60000 (30%)]	Loss: 0.005566	Center Loss: 0.001989
Train Epoch: 46 [19200/60000 (32%)]	Loss: 0.006844	Center Loss: 0.001812
Train Epoch: 46 [20480/60000 (34%)]	Loss: 0.004719	Center Loss: 0.001353
Train Epoch: 46 [21760/60000 (36%)]	Loss: 0.005824	Center Loss: 0.002235
Train Epoch: 46 [23040/60000 (38%)]	Loss: 0.005079	Center Loss: 0.001691
Train Epoch: 46 [24320/60000 (41%)]	Loss: 0.006419	Center Loss: 0.001858
Train Epoch: 46 [25600/60000 (43%)]	Loss: 0.006603	C

Train Epoch: 48 [33280/60000 (55%)]	Loss: 0.004255	Center Loss: 0.001170
Train Epoch: 48 [34560/60000 (58%)]	Loss: 0.003812	Center Loss: 0.001163
Train Epoch: 48 [35840/60000 (60%)]	Loss: 0.005187	Center Loss: 0.001351
Train Epoch: 48 [37120/60000 (62%)]	Loss: 0.005092	Center Loss: 0.001427
Train Epoch: 48 [38400/60000 (64%)]	Loss: 0.005740	Center Loss: 0.001861
Train Epoch: 48 [39680/60000 (66%)]	Loss: 0.006018	Center Loss: 0.001762
Train Epoch: 48 [40960/60000 (68%)]	Loss: 0.005586	Center Loss: 0.001680
Train Epoch: 48 [42240/60000 (70%)]	Loss: 0.006120	Center Loss: 0.002275
Train Epoch: 48 [43520/60000 (72%)]	Loss: 0.004537	Center Loss: 0.001640
Train Epoch: 48 [44800/60000 (75%)]	Loss: 0.005746	Center Loss: 0.001904
Train Epoch: 48 [46080/60000 (77%)]	Loss: 0.005002	Center Loss: 0.001412
Train Epoch: 48 [47360/60000 (79%)]	Loss: 0.004344	Center Loss: 0.001745
Train Epoch: 48 [48640/60000 (81%)]	Loss: 0.005080	Center Loss: 0.001350
Train Epoch: 48 [49920/60000 (83%)]	Loss: 0.005864	

Train Epoch: 50 [56320/60000 (94%)]	Loss: 0.005077	Center Loss: 0.001450
Train Epoch: 50 [57600/60000 (96%)]	Loss: 0.004366	Center Loss: 0.001022
Train Epoch: 50 [58880/60000 (98%)]	Loss: 0.006989	Center Loss: 0.002271

Test set: Average loss: 0.0001, Accuracy: 9881/10000 (99%)

Finished. Total elapsed time (h:m:s): 0:09:35
